In [1]:
import numpy as np
import spacy
nlp = spacy.load("en_core_web_lg")
import en_core_web_lg
import pandas as pd

from funcs.feature_transformation_funcs import *
import pylcs
from sklearn.feature_extraction.text import TfidfVectorizer


In [52]:
full_attributes = pd.read_csv("data/preprocessed data/full attributes as cols.csv")
full_preprocessed_data = pd.read_csv("data/preprocessed data/full preprocessed data.csv")
test_attributes = pd.read_csv("data/preprocessed data/test attributes as cols.csv")
test_preprocessed_data = pd.read_csv("data/preprocessed data/test preprocessed data.csv")

C:\Users\spiro\AppData\Local\Temp\ipykernel_3860\2332377633.py:2: DtypeWarning: Columns (6,9,10,11,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  full_preprocessed_data = pd.read_csv("data/preprocessed data/full preprocessed data.csv")


In [60]:
test_preprocessed_data.columns 

Index(['Unnamed: 0', 'id', 'product_uid_left', 'product_title', 'search_term',
       'relevance', 'product_description', 'search_term_stem',
       'product_title_stem', 'product_description_stem',
       'product_info_stemmed', 'product_info', 'product_uid_right', 'name',
       'value', 'all_values', 'search_term_and_all_attributes', 'product_uid',
       'Assembled Measurement (DxWxH)', 'Product Measurement (LxWxD)',
       'Product Measurement (DxWxH)', 'title_and_assembled_measurement',
       'title_and_measurement_LWD', 'title_and_measurement_DWH',
       'title_and_assembled_measurement_stem',
       'title_and_measurement_LWD_stem', 'title_and_measurement_DWH_stem',
       'Color Family', 'Color/Finish', 'Color Family Stem',
       'Color/Finish Stem', 'title_and_color', 'title_and_color_fam',
       'title_and_color_stem', 'title_and_color_fam_stem'],
      dtype='object')

In [34]:
print(full_attributes.shape)
print(full_preprocessed_data.shape)
print(test_attributes.shape)
print(test_preprocessed_data.shape)

(86261, 24)
(166693, 35)
(14, 24)
(400, 35)


In [35]:
# def calc_query_legth(df_all):
# 	# CALCULATE THE LENGTH OF THE SEARCH QUERY
# 	df_all['len_of_query'] = df_all['search_term_stem'].map(lambda x:0 if type(x) != str else len(x.split())).astype(np.int64)
 
# 	return df_all

In [2]:
# data = loaded_data
# calc_query_legth(data)
# count_common_words(data)
# calc_spacy_title_query(data)
# print('starting long spacy')
# calc_spacy_long(data)
# unstemmed_all, stemmed_all, counts_only, spacy_only = finalise_and_export(data)

In [116]:
def longest_subsequence(df_all, attributes):
    
    combined = pd.concat((df_all, attributes), axis = 1)
    
    df_all['lcs_title'] = combined.apply(lambda x: pylcs.lcs_sequence_length(str(x['search_term_stem']), str(x['product_title_stem'])), axis = 1)
    df_all['lcs_brand'] = combined.apply(lambda x: pylcs.lcs_sequence_length(str(x['search_term_stem']), str(x['MFG Brand Name'])), axis = 1)
    df_all['lcs_title_string'] = combined.apply(lambda x: pylcs.lcs_string_length(str(x['search_term_stem']), str(x['product_title_stem'])), axis = 1)
    df_all['lcs_brand_string'] = df_all.apply(lambda x: pylcs.lcs_string_length(str(x['search_term_stem']), str(x['MFG Brand Name'])), axis = 1)
    
    return df_all

In [35]:
def unique_str_common_word(str1, str2):
	if type(str1) == str and type(str2) == str:
		return sum(int(str2.find(word)>=0) for word in set(str1.split()))
	else:
		return

In [36]:
def unique_str_common_letters(str1, str2):
	if type(str1) == str and type(str2) == str:
		return sum(len(word) if word in str2 else 0 for word in set(str1.split()))
	else:
		return

In [52]:
'''The ratio of number of letters in the intersection of unique words in the query and title, description, and attributes yielded 3 more
 features (6). The same thing but with number of words instead of letters yielded another 3 (9)'''
 
def count_unique_common_words(df_all):
    df_all['unique_word_in_title_stemmed'] = df_all['product_info_stemmed'].map(lambda x:unique_str_common_word(x.split('\t')[0],x.split('\t')[1]) if type(x) == str else 0)
    df_all['unique_word_in_description_stemmed'] = df_all['product_info_stemmed'].map(lambda x:unique_str_common_word(x.split('\t')[0],x.split('\t')[2]) if type(x) == str else 0)

    # uncomment to include unstemmed
    # df_all['unique_word_in_title'] = df_all['product_info'].map(lambda x:unique_str_common_word(x.split('\t')[0],x.split('\t')[1]) if type(x) == str else 0)
    # df_all['unique_word_in_description'] = df_all['product_info'].map(lambda x:unique_str_common_word(x.split('\t')[0],x.split('\t')[2]) if type(x) == str else 0)
# 
    # CODE BLOCK 4: COMPUTING NEW COUNT FEATURES
    # ADDING ATTRIBUTES - COUNT OF PRODUCT ATTRIBUTES IN THE SEARCH TERM
    df_all['unique_word_in_attributes'] = df_all['search_term_and_all_attributes'].map(lambda x:unique_str_common_word(x.split('\t')[0],x.split('\t')[2]) if type(x)==str else 0)

    return df_all

In [53]:
df_all = count_unique_common_words(test_preprocessed_data)

In [54]:
def count_unique_common_letter(df_all):

    df_all['unique_letter_in_title_stemmed'] = df_all['product_info_stemmed'].map(lambda x:unique_str_common_letters(x.split('\t')[0],x.split('\t')[1]) if type(x) == str else 0)
    df_all['unique_letter_in_description_stemmed'] = df_all['product_info_stemmed'].map(lambda x:unique_str_common_letters(x.split('\t')[0],x.split('\t')[2]) if type(x) == str else 0)

    # uncomment to include unstemmed
    # df_all['unique_word_in_title'] = df_all['product_info'].map(lambda x:unique_str_common_letters(x.split('\t')[0],x.split('\t')[1]) if type(x) == str else 0)
    # df_all['unique_word_in_description'] = df_all['product_info'].map(lambda x:unique_str_common_letters(x.split('\t')[0],x.split('\t')[2]) if type(x) == str else 0)

    # CODE BLOCK 4: COMPUTING NEW COUNT FEATURES
    # ADDING ATTRIBUTES - COUNT OF PRODUCT ATTRIBUTES IN THE SEARCH TERM
    df_all['unique_letter_in_attributes'] = df_all['search_term_and_all_attributes'].map(lambda x:unique_str_common_letters(x.split('\t')[0],x.split('\t')[2]) if type(x)==str else 0)

    return df_all

In [55]:
df_all = count_unique_common_letter(df_all)

In [67]:
def calc_frequency_ratios(df_all):
    df_all['len_search_term_stem_word'] = df_all['search_term_stem'].map(lambda x: len(x.split()))
    df_all['len_search_term_stem_letter'] = df_all['search_term_stem'].map(lambda x: len(x))
    
    df_all['search_title_uniqe_words'] = df_all['unique_word_in_title_stemmed']/ df_all['len_search_term_stem_word']
    df_all['search_description_uniqe_words'] = df_all['unique_word_in_description_stemmed']/ df_all['len_search_term_stem_word']
    df_all['search_attributes_uniqe_words'] = df_all['unique_word_in_attributes']/ df_all['len_search_term_stem_word']
    
    df_all['search_title_uniqe_letters'] = df_all['unique_letter_in_title_stemmed']/ df_all['len_search_term_stem_letter']
    df_all['search_description_uniqe_letters'] = df_all['unique_letter_in_description_stemmed']/ df_all['len_search_term_stem_letter']
    df_all['search_attributes_uniqe_letters'] = df_all['unique_letter_in_attributes']/ df_all['len_search_term_stem_letter']
    
    return df_all

In [68]:
calc_frequency_ratios(df_all)

In [18]:
geeks =pd.DataFrame({
    'query':['Ben','Steve', 'Data Scientists'],
    'description':[
        'Ben studies about computer in Computer Lab.',
        'Steve teaches at Brown University.',
        'Data Scientists work on large datasets.'
        ]
})

In [51]:
a = np.array([[1,1,1,1],[2,2,2,2]])
b = dict({
    'dog':2,
    'cat':3,
    'deer':1,
    'bull':0
})
c = sorted(b, key = lambda x: b[x])
d = [len(word) for word in c]
e = a*d
e
b

{'dog': 2, 'cat': 3, 'deer': 1, 'bull': 0}

In [21]:
def get_tfidf_score(df_all):
        
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_scores = tfidf_vectorizer.fit_transform(df_all['description'])

    sorted_vocab = sorted(tfidf_vectorizer.vocabulary_, key = lambda x: tfidf_vectorizer.vocabulary_)
    df_all['word_len_weighted_scores'] = pd.DataFrame(tfidf_scores * sorted_vocab)
    
    df_all['weighted_tfidf'] = df_all.apply(lambda x: sum[x['word_len_weighted_scores'][word] for word in x['search_term']], axis = 1)
    
    df_all.drop('word_len_weighted_scores')
    
    return df_all   

array([[0.33333333, 0.        , 0.33333333, 0.        , 0.66666667,
        0.        , 0.        , 0.33333333, 0.33333333, 0.        ,
        0.        , 0.        , 0.        , 0.33333333, 0.        ,
        0.        , 0.        ],
       [0.        , 0.4472136 , 0.        , 0.4472136 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.4472136 , 0.        , 0.4472136 ,
        0.4472136 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.40824829, 0.40824829, 0.        , 0.        , 0.40824829,
        0.40824829, 0.40824829, 0.        , 0.        , 0.        ,
        0.        , 0.40824829]])

In [22]:
print(tfidf_vectorizer.vocabulary_)

{'ben': 2, 'studies': 13, 'about': 0, 'computer': 4, 'in': 7, 'lab': 8, 'steve': 12, 'teaches': 14, 'at': 1, 'brown': 3, 'university': 15, 'data': 5, 'scientists': 11, 'work': 16, 'on': 10, 'large': 9, 'datasets': 6}
